In [1]:
import os
os.chdir('../')
%pwd

'd:\\Projects\\DeepFake Voice Recognition\\audio-deepfake-detection'

In [2]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    evaluation_results_dir: Path
    trained_model_dir: Path
    data_dir: Path
    metrics: list

In [3]:
from audioClassifier.constants import *
from audioClassifier.utils.common import open_yaml_file, create_directories, write_to_json

In [4]:
class ConfigManager:
    def __init__(self, config_file = CONFIG_PATH, params_file = PARAMS_PATH):
        self.config = open_yaml_file(config_file)
        self.params = open_yaml_file(params_file)
        
        create_directories([self.config.artifacts_root])
        
    def read_model_eval_config(self) -> ModelEvaluationConfig:
        model_eval = self.config.model_evaluation
        
        create_directories([self.config.model_evaluation.root_dir])
        
        model_eval_config = ModelEvaluationConfig(
            root_dir = Path(model_eval.root_dir),
            evaluation_results_dir = Path(model_eval.evaluation_results_dir),
            trained_model_dir = Path(self.config.model_training.trained_model_dir),
            data_dir = Path(self.config.data_validation.data_path),
            metrics = self.params.metrics
        )
        
        return model_eval_config

In [5]:
import numpy as np
from tensorflow.keras.models import load_model
from audioClassifier import logger

2024-07-25 11:08:57,946 - module_wrapper.py - WARNING - From d:\Projects\DeepFake Voice Recognition\audio-deepfake-detection\venv\lib\site-packages\keras\src\losses.py:2976: The name tf.losses.sparse_softmax_cross_entropy is deprecated. Please use tf.compat.v1.losses.sparse_softmax_cross_entropy instead.



In [6]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config
        
    def _load_val_data(self):
        try:
            X_test = np.load(self.config.data_dir / 'X_test.npy')
            y_test = np.load(self.config.data_dir / 'y_test.npy')
            return X_test, y_test
        except Exception as e:
            logger.error(f"Error loading validation data: {e}")
            raise e
    
    def _load_trained_model(self):
        try:
            model = load_model(str(self.config.trained_model_dir))
            return model
        except Exception as e:
            logger.error(f"Error loading the trained model: {e}")
            raise e
        
    def _parse_metrics(self, metrics_list):
        parsed_metrics = ['loss'] + metrics_list
        return parsed_metrics
    
    def evaluate_model(self):
        X_test, y_test = self._load_val_data()
        model = self._load_trained_model()
        metrics_list = self._parse_metrics(self.config.metrics)
        
        try:
            score = model.evaluate(X_test, y_test)
            eval_scores = dict(zip(metrics_list, score))
            write_to_json(eval_scores, self.config.evaluation_results_dir)
            logger.info(f"Model evaluated successfully and results saved to {self.config.evaluation_results_dir}.")
        except Exception as e:
            logger.error(f"Error evaluating the model: {e}")
            raise e

In [7]:
try:
    config = ConfigManager()
    model_evaluation_config = config.read_model_eval_config()
    model_evaluation = ModelEvaluation(config = model_evaluation_config)
    model_evaluation.evaluate_model()
except Exception as e:
    raise e

2024-07-25 11:08:58,566 - common.py - INFO - YAML file 'config\config.yaml' was loaded successfully.
2024-07-25 11:08:58,568 - common.py - INFO - YAML file 'params.yaml' was loaded successfully.
2024-07-25 11:08:58,569 - common.py - INFO - Directory 'artifacts' created successfully or already exists.
2024-07-25 11:08:58,570 - common.py - INFO - Directory 'artifacts/model_evaluation' created successfully or already exists.
2024-07-25 11:08:58,921 - module_wrapper.py - WARNING - From d:\Projects\DeepFake Voice Recognition\audio-deepfake-detection\venv\lib\site-packages\keras\src\backend.py:1398: The name tf.executing_eagerly_outside_functions is deprecated. Please use tf.compat.v1.executing_eagerly_outside_functions instead.

2024-07-25 11:08:59,122 - module_wrapper.py - WARNING - From d:\Projects\DeepFake Voice Recognition\audio-deepfake-detection\venv\lib\site-packages\keras\src\layers\normalization\batch_normalization.py:979: The name tf.nn.fused_batch_norm is deprecated. Please use t

19/19 [==============================] - 5s 204ms/step - loss: 0.1602 - accuracy: 0.9665 - auc: 0.9896 - recall: 0.9693


2024-07-25 11:09:12,434 - common.py - INFO - Data successfully written and saved to JSON file: 'artifacts\model_evaluation\evaluation_scores.json'.
2024-07-25 11:09:12,435 - 1737105841.py - INFO - Model evaluated successfully and results saved to artifacts\model_evaluation\evaluation_scores.json.
